In [48]:
import nltk
from nltk.corpus import treebank

sents = treebank.parsed_sents()

ratio = 0.9
train_size = int(len(sents) * ratio)
sents_train, sents_test = sents[:train_size], sents[train_size:]

# productions need to be filtered according to Horacio criteria
productions_train = set(production for sent in sents_train
                        for production in sent.productions())

In [52]:
sents[0].productions()

[S -> NP-SBJ VP .,
 NP-SBJ -> NP , ADJP ,,
 NP -> NNP NNP,
 NNP -> 'Pierre',
 NNP -> 'Vinken',
 , -> ',',
 ADJP -> NP JJ,
 NP -> CD NNS,
 CD -> '61',
 NNS -> 'years',
 JJ -> 'old',
 , -> ',',
 VP -> MD VP,
 MD -> 'will',
 VP -> VB NP PP-CLR NP-TMP,
 VB -> 'join',
 NP -> DT NN,
 DT -> 'the',
 NN -> 'board',
 PP-CLR -> IN NP,
 IN -> 'as',
 NP -> DT JJ NN,
 DT -> 'a',
 JJ -> 'nonexecutive',
 NN -> 'director',
 NP-TMP -> NNP CD,
 NNP -> 'Nov.',
 CD -> '29',
 . -> '.']

In [46]:
from nltk.grammar import CFG, Nonterminal
treebank_grammar = CFG(Nonterminal('S'), list(productions_train))

In [47]:
from nltk.grammar import PCFG, induce_pcfg
treebank_prob_grammar = induce_pcfg(Nonterminal('S'), list(productions_train))

In [41]:
sents_test = treebank.sents()[train_size:]

In [44]:
# Load the grammar into the ChartParser.
cp = nltk.ChartParser(treebank_prob_grammar)
# Generate and print the nbest_parse from the grammar given the sentence tokens.
cp.parse(sents_test[1])

KeyboardInterrupt: 